# Notebook 13: Dimensional Thermal Convection

This notebook demonstrates a realistic thermal convection model using physical dimensions and units. We solve the coupled Stokes flow and thermal diffusion equations with realistic Earth parameters.

## Physical Problem

We model thermal convection in Earth's mantle using dimensional equations:

**Momentum Balance (Stokes Flow):**
$$
-\nabla \cdot
    \left[
            \frac{\eta}{2}\left( \nabla \mathbf{u} + \nabla \mathbf{u}^T \right) -  p \mathbf{I} \right] =
     -\rho_0 \alpha (T - T_{ref}) \mathbf{g} 
$$
$$
\nabla \cdot \mathbf{u} = 0
$$

**Thermal Evolution:**
$$
\frac{\partial T}{\partial t} + \mathbf{u}\cdot\nabla T = \kappa \nabla^2 T 
$$

where:
- $\eta$ = dynamic viscosity (Pa⋅s)
- $\rho_0$ = reference density (kg/m³)
- $\alpha$ = thermal expansion coefficient (1/K)
- $T$ = temperature (K)
- $\kappa$ = thermal diffusivity (m²/s)
- $\mathbf{g}$ = gravitational acceleration (m/s²)

In [1]:
import numpy as np
import sympy
import underworld3 as uw

# Set up realistic physical parameters for Earth's mantle
print("=== DIMENSIONAL MANTLE CONVECTION MODEL ===")
print("\nPhysical Parameters:")

# Geometry (mantle section)
radius_inner = 3500e3      # m (core-mantle boundary)
radius_outer = 6400e3      # m (near surface)
layer_thickness = radius_outer - radius_inner  # 2900 km

print(f"Inner radius: {radius_inner/1000:.0f} km")
print(f"Outer radius: {radius_outer/1000:.0f} km")
print(f"Layer thickness: {layer_thickness/1000:.0f} km")

# Material properties
rho_0 = 3300.0            # kg/m³ (reference density)
alpha = 3e-5              # 1/K (thermal expansion)
eta_0 = 1e21              # Pa⋅s (reference viscosity)
kappa = 1e-6              # m²/s (thermal diffusivity)
g = 9.81                  # m/s² (gravity)
T_ref = 1600              # K (reference temperature)
Delta_T = 1300            # K (temperature drop across layer)

print(f"\nMaterial Properties:")
print(f"Reference density: {rho_0:.0f} kg/m³")
print(f"Thermal expansion: {alpha:.1e} 1/K")
print(f"Reference viscosity: {eta_0:.1e} Pa⋅s")
print(f"Thermal diffusivity: {kappa:.1e} m²/s")
print(f"Temperature drop: {Delta_T:.0f} K")

# Dimensional analysis
Ra = rho_0 * g * alpha * Delta_T * layer_thickness**3 / (eta_0 * kappa)
thermal_time = layer_thickness**2 / kappa
convective_velocity = kappa / layer_thickness

print(f"\nDimensional Analysis:")
print(f"Rayleigh number: Ra = {Ra:.2e}")
print(f"Thermal diffusion time: {thermal_time/(1e6*365.25*24*3600):.0f} Myr")
print(f"Characteristic velocity: {convective_velocity*100*365.25*24*3600:.1f} cm/year")

=== DIMENSIONAL MANTLE CONVECTION MODEL ===

Physical Parameters:
Inner radius: 3500 km
Outer radius: 6400 km
Layer thickness: 2900 km

Material Properties:
Reference density: 3300 kg/m³
Thermal expansion: 3.0e-05 1/K
Reference viscosity: 1.0e+21 Pa⋅s
Thermal diffusivity: 1.0e-06 m²/s
Temperature drop: 1300 K

Dimensional Analysis:
Rayleigh number: Ra = 3.08e+07
Thermal diffusion time: 266497 Myr
Characteristic velocity: 0.0 cm/year


## Mesh Setup

Create an annulus mesh representing the mantle section:

In [2]:
# Create annulus mesh with realistic dimensions
cell_size = 100e3  # 100 km resolution

mesh = uw.meshing.Annulus(
    radiusInner=radius_inner,
    radiusOuter=radius_outer, 
    cellSize=cell_size,
    qdegree=3,
    verbose=True
)

print(f"\nMesh created with {mesh.points.shape[0]} nodes")
print(f"Cell size: {cell_size/1000:.0f} km")
print(f"Mesh type: {mesh.parameters.mesh_type}")
print(f"Radial extent: {mesh.parameters.radial_extent/1000:.0f} km")

# Coordinate systems
x, y = mesh.CoordinateSystem.X
r, theta = mesh.CoordinateSystem.xR
unit_r = mesh.CoordinateSystem.unit_e_0  # radial unit vector

Constructing UW mesh from gmsh .meshes/uw_annulus_ro6400000.0_ri3500000.0_csize100000.0.msh
Mesh refinement levels: None
Mesh coarsening levels: None
PETSc dmplex set-up complete
PETSc spatial discretisation
[0] PETScDS - (re) initialised
PETScFE - (re) initialised
PETSc DM - coordinates
UW kD-Tree
UW kD-Tree - constructed
Mesh Spatial Discretisation Complete
Populating mesh coordinates CoordinateSystemType.CYLINDRICAL2D

Mesh created with 10948 nodes
Cell size: 100 km
Mesh type: Annulus
Radial extent: 2900 km


## Variables and Equations

Define mesh variables for velocity, pressure, and temperature with appropriate degrees:

In [3]:
# Create mesh variables
velocity = uw.discretisation.MeshVariable("velocity", mesh, 2, degree=2)
pressure = uw.discretisation.MeshVariable("pressure", mesh, 1, degree=1)
temperature = uw.discretisation.MeshVariable("temperature", mesh, 1, degree=3)

## Stokes Flow Solver

Set up the Stokes solver with dimensional buoyancy force:

In [4]:
# Create Stokes solver
stokes = uw.systems.Stokes(
    mesh,
    velocityField=velocity,
    pressureField=pressure,
)

# Dimensional buoyancy force: -ρ₀ α (T - T_ref) g ê_r
buoyancy_force = -rho_0 * alpha * (temperature - T_ref) * g * unit_r
stokes.bodyforce = buoyancy_force

print(f"Buoyancy force expression: {buoyancy_force}")

# Constitutive model with dimensional viscosity
stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel
stokes.constitutive_model.Parameters.shear_viscosity_0 = eta_0

print(f"Reference viscosity: {eta_0:.2e} Pa⋅s")

# Solver settings
stokes.tolerance = 1.0e-4
stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"

# Free-slip boundary conditions (zero radial velocity)
# Apply natural BC: penalize radial velocity component
penalty_factor = 1e6  # Large penalty for free-slip

stokes.add_natural_bc(
    penalty_factor * unit_r.dot(velocity) * unit_r, "Upper"
)
stokes.add_natural_bc(
    penalty_factor * unit_r.dot(velocity) * unit_r, "Lower"
)

print(f"\nBoundary conditions:")
print(f"Upper/Lower: Free-slip (v_r = 0, penalty = {penalty_factor:.1e})")

Buoyancy force expression: Matrix([[N.x*(1553.904 - 0.97119*{ \hspace{ 0.04pt } {temperature} }(N.x, N.y))/sqrt(N.x**2 + N.y**2), N.y*(1553.904 - 0.97119*{ \hspace{ 0.04pt } {temperature} }(N.x, N.y))/sqrt(N.x**2 + N.y**2)]])
Reference viscosity: 1.00e+21 Pa⋅s

Boundary conditions:
Upper/Lower: Free-slip (v_r = 0, penalty = 1.0e+06)


## Advection-Diffusion Solver

Set up the advection-diffusion solver for temperature evolution:

In [5]:
# Create advection-diffusion solver
thermal_solver = uw.systems.AdvDiffusion(
    mesh,
    u_Field=temperature,
    V_fn=velocity,  # Velocity from Stokes solver
    order=2,
    verbose=True,
)

# Constitutive model with dimensional diffusivity
thermal_solver.constitutive_model = uw.constitutive_models.DiffusionModel
thermal_solver.constitutive_model.Parameters.diffusivity = kappa

print(f"Thermal diffusivity: {kappa:.2e} m²/s")

# Temperature boundary conditions
T_surface = T_ref - Delta_T/2   # 950 K at surface
T_cmb = T_ref + Delta_T/2       # 2250 K at core-mantle boundary

thermal_solver.add_dirichlet_bc(T_cmb, "Lower")    # Hot at bottom
thermal_solver.add_dirichlet_bc(T_surface, "Upper") # Cool at top

print(f"\nTemperature boundary conditions:")
print(f"Surface (Upper): {T_surface:.0f} K ({T_surface-273:.0f} °C)")
print(f"CMB (Lower): {T_cmb:.0f} K ({T_cmb-273:.0f} °C)")
print(f"Temperature drop: {T_cmb - T_surface:.0f} K")

# Solver tolerances
thermal_solver.petsc_options.setValue("snes_rtol", 1e-4)
thermal_solver.petsc_options.setValue("ksp_rtol", 1e-5)

Delayed callback error: maximum recursion depth exceeded
Delayed callback error: maximum recursion depth exceeded
Delayed callback error: maximum recursion depth exceeded
Delayed callback error: maximum recursion depth exceeded


Thermal diffusivity: 1.00e-06 m²/s

Temperature boundary conditions:
Surface (Upper): 950 K (677 °C)
CMB (Lower): 2250 K (1977 °C)
Temperature drop: 1300 K


## Initial Conditions

Set up realistic initial temperature field:

In [6]:
# Initial temperature: conductive + small perturbation
# Conductive profile: linear with radius
conductive_profile = T_surface + (T_cmb - T_surface) * (radius_outer - r) / (radius_outer - radius_inner)

# Add small thermal perturbation to trigger convection
perturbation_amplitude = 50  # K
thermal_perturbation = perturbation_amplitude * sympy.sin(3 * theta) * \
                      sympy.cos(np.pi * (r - radius_inner) / (radius_outer - radius_inner))

initial_temperature = conductive_profile + thermal_perturbation

# Set initial condition
with uw.synchronised_array_update():
    temperature.array[...] = uw.function.evaluate(initial_temperature, temperature.coords)

# Check initial temperature statistics
temp_stats = temperature.stats()
print(f"Initial temperature statistics:")
print(f"Min: {temp_stats['min']:.0f} K ({temp_stats['min']-273:.0f} °C)")
print(f"Max: {temp_stats['max']:.0f} K ({temp_stats['max']-273:.0f} °C)")
print(f"Mean: {temp_stats['mean']:.0f} K ({temp_stats['mean']-273:.0f} °C)")
print(f"Perturbation amplitude: {perturbation_amplitude} K")

Initial temperature statistics:
Min: 900 K (627 °C)
Max: 3819 K (3546 °C)
Mean: 1533 K (1260 °C)
Perturbation amplitude: 50 K


## Initial Stokes Solve

Solve for initial velocity field:

In [7]:
# Solve initial Stokes problem
print("Solving initial Stokes flow...")
stokes.solve(zero_init_guess=True)

# Check velocity statistics (will implement vector stats)
print("\nInitial velocity field solved")
print(f"Velocity variable type: {velocity.num_components}-component")

# Calculate velocity magnitude manually for now
coords = mesh.data
v_x = velocity.array[:, 0, 0].flatten()
v_y = velocity.array[:, 0, 1].flatten()
v_magnitude = np.sqrt(v_x**2 + v_y**2)

print(f"Velocity magnitude range: {v_magnitude.min():.2e} to {v_magnitude.max():.2e} m/s")
print(f"Max velocity: {v_magnitude.max()*100*365.25*24*3600:.2f} cm/year")

Solving initial Stokes flow...

Initial velocity field solved
Velocity variable type: 2-component
Velocity magnitude range: 0.00e+00 to 0.00e+00 m/s
Max velocity: 0.00 cm/year


## Time-stepping Loop

Evolve the coupled system through time:

In [ ]:
# Time-stepping parameters
max_steps = 20
time_step = 0
elapsed_time = 0.0  # seconds

print("=== TIME-STEPPING LOOP ===")
print(f"Maximum steps: {max_steps}")

# Storage for monitoring
time_history = []
max_velocity_history = []
mean_temperature_history = []

for step in range(max_steps):
    # Solve Stokes flow with current temperature
    stokes.solve(zero_init_guess=False)
    
    # Estimate stable time step
    dt_estimate = thermal_solver.estimate_dt().squeeze()
    dt = 2.0 * dt_estimate  # Use 2x estimated dt
    
    # Solve thermal evolution
    thermal_solver.solve(timestep=dt, zero_init_guess=False, verbose=False)
    
    # Update time
    time_step += 1
    elapsed_time += dt
    
    # Monitor solution
    if time_step % 5 == 0 or step < 5:
        # Get statistics
        temp_stats = temperature.stats()
        
        # Calculate velocity magnitude
        v_x = velocity.array[:, 0, 0].flatten()
        v_y = velocity.array[:, 0, 1].flatten()
        v_mag = np.sqrt(v_x**2 + v_y**2)
        max_vel = v_mag.max()
        
        # Store history
        time_history.append(elapsed_time)
        max_velocity_history.append(max_vel)
        mean_temperature_history.append(temp_stats['mean'])
        
        # Convert to convenient units
        time_myr = elapsed_time / (1e6 * 365.25 * 24 * 3600)
        vel_cmyr = max_vel * 100 * 365.25 * 24 * 3600
        
        print(f"Step {time_step:3d}: t = {time_myr:.3f} Myr, "
              f"dt = {dt_estimate:.2e} s, "
              f"max_vel = {vel_cmyr:.2f} cm/yr, "
              f"mean_T = {temp_stats['mean']:.0f} K")

print(f"\nTime-stepping completed!")
print(f"Final time: {elapsed_time/(1e6*365.25*24*3600):.3f} Myr")
print(f"Final max velocity: {max_velocity_history[-1]*100*365.25*24*3600:.2f} cm/year")

=== TIME-STEPPING LOOP ===
Maximum steps: 20


## Analysis and Results

Analyze the final solution:

In [ ]:
# Final solution analysis
print("=== FINAL SOLUTION ANALYSIS ===")

# Temperature statistics
temp_stats = temperature.stats()
print(f"\nTemperature field:")
print(f"  Min: {temp_stats['min']:.0f} K ({temp_stats['min']-273:.0f} °C)")
print(f"  Max: {temp_stats['max']:.0f} K ({temp_stats['max']-273:.0f} °C)")
print(f"  Mean: {temp_stats['mean']:.0f} K ({temp_stats['mean']-273:.0f} °C)")
print(f"  RMS: {temp_stats['rms']:.0f} K")

# Velocity analysis
coords = mesh.data
v_x = velocity.array[:, 0, 0].flatten()
v_y = velocity.array[:, 0, 1].flatten()
v_magnitude = np.sqrt(v_x**2 + v_y**2)

print(f"\nVelocity field:")
print(f"  Min magnitude: {v_magnitude.min():.2e} m/s ({v_magnitude.min()*100*365.25*24*3600:.3f} cm/yr)")
print(f"  Max magnitude: {v_magnitude.max():.2e} m/s ({v_magnitude.max()*100*365.25*24*3600:.2f} cm/yr)")
print(f"  Mean magnitude: {v_magnitude.mean():.2e} m/s ({v_magnitude.mean()*100*365.25*24*3600:.2f} cm/yr)")

# Compare with scaling estimates
print(f"\nComparison with scaling:")
print(f"  Characteristic velocity: {convective_velocity*100*365.25*24*3600:.1f} cm/yr")
print(f"  Actual max velocity: {v_magnitude.max()*100*365.25*24*3600:.1f} cm/yr")
print(f"  Velocity ratio: {v_magnitude.max()/convective_velocity:.1f}")

# Heat flux calculation (dimensional)
temperature_gradient = mesh.vector.gradient(temperature)
radial_heat_flux = -thermal_solver.constitutive_model.Parameters.diffusivity * \
                   temperature_gradient.dot(unit_r)

# Evaluate heat flux at boundaries
print(f"\nHeat flux analysis:")
print(f"  Thermal diffusivity: {kappa:.2e} m²/s")
print(f"  Heat flux expression: -κ ∇T⋅ê_r")

# Nusselt number (heat flux enhancement)
conductive_flux = kappa * Delta_T / layer_thickness
print(f"  Conductive heat flux: {conductive_flux:.2e} W/m² (if we add density and heat capacity)")

## Summary

### Physical Insights
- **Rayleigh number**: ~10⁸ for vigorous convection
- **Convection velocities**: Several cm/year (realistic for mantle)
- **Temperature structure**: Develops thermal boundary layers
- **Time scales**: Evolution over millions of years

### Model Applications

This approach enables:
- **Geodynamic modeling**: Realistic Earth simulations
- **Parameter studies**: Effects of viscosity, thermal properties
- **Comparative planetology**: Different planetary conditions
- **Multi-physics coupling**: Add chemical evolution, phase changes

The dimensional approach provides direct physical insight while maintaining numerical stability through proper scaling analysis.